In [ ]:
import json
import os
import pandas as pd
import pickle

from utils.data_dirs import data_dirs
from utils.index import index
from utils.unique import unique

In [ ]:
EXPERIMENT = False

In [ ]:
base_dir, output_dir, folder, EXPERIMENT_DATA, suffix, *_ = data_dirs(EXPERIMENT)
print(base_dir)
print(output_dir)
print(folder)
print(EXPERIMENT_DATA)
print(suffix)

In [ ]:
with open(os.path.join(base_dir, folder, 'category.json')) as f:
    category_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample.json')) as f:
    sample_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample_annotation.json')) as f:
    sample_annotation_json = json.load(f)

with open(os.path.join(base_dir, folder, 'instance.json')) as f:
    instance_json = json.load(f)

with open(os.path.join(base_dir, folder, 'scene.json')) as f:
    scene_json = json.load(f)

with open(os.path.join(base_dir, folder, 'attribute.json')) as f:
    attribute_json = json.load(f)

with open(os.path.join(base_dir, folder, 'log.json')) as f:
    log_json = json.load(f)

In [ ]:
log_filter = [
    {
        "log_token": l["token"],
        "location": l["location"]
    }
    for l in log_json
]
len(log_filter)

In [ ]:
pd.DataFrame(log_filter)

In [ ]:
sample_filter = [
    {
        "sample_token": s["token"],
        "timestamp": s["timestamp"],
        "scene_token": s["scene_token"]
    }
    for s in sample_json
]
len(sample_filter)

In [ ]:
pd.DataFrame(sample_filter)

In [ ]:
scene_filter = [
    {
        "scene_token": s["token"],
        "log_token": s["log_token"],
        "name": s["name"],
    }
    for s in scene_json
]
len(scene_filter)

In [ ]:
pd.DataFrame(scene_filter)

In [ ]:
sample_annotation_filter = [
    sa
    for sa in sample_annotation_json
]
len(sample_annotation_filter)

In [ ]:
pd.DataFrame(sample_annotation_filter)

In [ ]:
instance_tokens = unique(sample_annotation_filter, 'instance_token')
instance_filter = [
    {
        'instance_token': i['token'],
        'category_token': i['category_token']
    }
    for i in instance_json
    if i['token'] in instance_tokens
]
len(instance_filter)

In [ ]:
pd.DataFrame(instance_filter)

In [ ]:
category_tokens = unique(instance_filter, 'category_token')
category_filter = [
    {
        'category_token': c['token'],
        'category': c['name']
    }
    for c in category_json
    if c['token'] in category_tokens
]
len(category_filter)

In [ ]:
instance_map = index(instance_filter, 'instance_token')
category_map = index(category_filter, 'category_token')
sample_map = index(sample_filter, 'sample_token')
log_map = index(log_filter, 'log_token')
scene_map = index(scene_filter, 'scene_token')

def sa_map(sa):
    instance = instance_map[sa['instance_token']]
    category = category_map[instance['category_token']]
    sample = sample_map[sa['sample_token']]
    scene = scene_map[sample['scene_token']]
    log = log_map[scene['log_token']]
    ret = {
        **sa,
        **instance,
        **category,
        **sample,
        **scene,
        **log,
    }
    
    del ret['token']
    del ret['visibility_token']
    del ret['attribute_tokens']
    del ret['prev']
    del ret['next']
    del ret['num_lidar_pts']
    del ret['num_radar_pts']
    del ret['category_token']
    del ret['scene_token']
    del ret['log_token']
    
    return ret

sample_annotation_res = [*map(sa_map, sample_annotation_filter)]
len(sample_annotation_res)

In [ ]:
sample_annotation_final = pd.DataFrame(sample_annotation_res)
sample_annotation_final[:1]

In [ ]:
locations = {}

for a in sample_annotation_res:
    location = a['location']
    if location not in locations:
        locations[location] = []
    locations[location].append(a)

In [ ]:
for location, annotations in locations.items():
    base = os.path.join(output_dir, 'partition', location)
    if not os.path.exists(base):
        os.makedirs(base)
    with open(os.path.join(base, 'ground_truth_annotation.pickle'), "wb") as f:
        pickle.dump(annotations, f)
    with open(os.path.join(base, 'ground_truth_annotation.json'), "w") as f:
        json.dump(annotations, f)